# YOLOv8 model evaluation

In [ ]:
from roboflow import Roboflow
import os

os.environ["DATASET_DIRECTORY"] = "C:/content/datasets"

rf = Roboflow(api_key="feG2z6aSS9JkkauzQ8Qo")
project = rf.workspace("lidar-object-detection").project("lidar-human-detection")
dataset = project.version(2).download("yolov8")

In [51]:
import cv2
import numpy as np
from ouster import client
from ouster import pcap
from ultralytics import YOLO
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from contextlib import closing
from collections import defaultdict


def avg_time_of_detection(weight_path):
    detection_time = []
    model = YOLO(weight_path)

    metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
    pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

    with open(metadata_path, 'r') as f:
        metadata = client.SensorInfo(f.read())

    pcap_file = pcap.Pcap(pcap_path, metadata)
    xyz_lut = client.XYZLut(metadata)

    with closing(Scans(pcap_file)) as scans:
        i = 0
        for scan in scans:
            i += 1
            sig_field = scan.field(client.ChanField.SIGNAL)
            sig_destaggered = destagger(metadata, sig_field)
            scaling_factor = 0.004
            scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
            signal_image = scaled_arr.astype(np.uint8)
            combined_img = np.dstack((signal_image, signal_image, signal_image))

            results = model.predict(source=combined_img, imgsz=1024, tracker='bytetrack.yaml', verbose=False)

            speed = results[0].speed
            time = speed['preprocess'] + speed['inference'] + speed['postprocess']
            detection_time.append(time)
            
            if i >= 100:   
                break
    return sum(detection_time)/len(detection_time)

In [46]:
avg_time = avg_time_of_detection('../weights/best_3000_s.pt')
print(avg_time)

61.05890250205994


In [52]:
from ultralytics import YOLO
import os


directory = '../weights/'

detection_results = {}
 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        # print(f)
        avg_time = avg_time_of_detection(str(f))
        print(avg_time)
        model = YOLO(f)
        metrics = model.val()
        formatted_results = {}
        for key, val in metrics.results_dict.items():
            if key != 'fitness':
                formatted_results[key[8:-3]] = val
            else:
                formatted_results[key] = val
        detection_results[f] = [avg_time, formatted_results]

32.429986000061035


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:06<00:00,  2.48it/s]
                   all        269        662      0.885      0.866      0.924       0.55
Speed: 0.3ms preprocess, 21.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val6


34.09240245819092


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:06<00:00,  2.49it/s]
                   all        269        662       0.87        0.9      0.934      0.571
Speed: 0.3ms preprocess, 21.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val7


63.93436908721924


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:19<00:00,  1.12s/it]
                   all        269        662      0.891      0.869      0.933      0.582
Speed: 0.3ms preprocess, 64.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val8


59.524595737457275


Ultralytics YOLOv8.0.208  Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800HS with Radeon Graphics)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\content\datasets\lidar-human-detection-2\valid\labels.cache... 269 images, 47 backgrounds, 0 corrupt: 100%|██████████| 269/269 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:15<00:00,  1.09it/s]
                   all        269        662       0.86      0.902      0.933      0.575
Speed: 0.3ms preprocess, 53.2ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val9


In [54]:
for key, val in detection_results.items():
    print(f"{key} :  {val[0]} ms, {val[1]}")

../weights/best_3000_n.pt :  32.429986000061035 ms, {'precision': 0.8852701575502824, 'recall': 0.8655589123867069, 'mAP50': 0.9244299681155909, 'mAP50-95': 0.5504321634699545, 'fitness': 0.5878319439345181}
../weights/best_3000_n_200.pt :  34.09240245819092 ms, {'precision': 0.8700962954286553, 'recall': 0.9004879694639911, 'mAP50': 0.9344139833233811, 'mAP50-95': 0.5709837026104664, 'fitness': 0.6073267306817579}
../weights/best_3000_s.pt :  63.93436908721924 ms, {'precision': 0.8909035734398139, 'recall': 0.8685800604229608, 'mAP50': 0.9328926085129733, 'mAP50-95': 0.5824135092485827, 'fitness': 0.6174614191750217}
../weights/best_3000_s_200.pt :  59.524595737457275 ms, {'precision': 0.8602582485491604, 'recall': 0.9020207590668169, 'mAP50': 0.9334241611932032, 'mAP50-95': 0.575082108600139, 'fitness': 0.6109163138594453}
